In [0]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects

In [0]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!unzip drive-download-20191201T013728Z-001.zip

Archive:  drive-download-20191201T013728Z-001.zip
  inflating: testOutputPredictions.npy  
  inflating: trainOutputPredictions.npy  
  inflating: testInputImages.npy     
  inflating: triplets_dataset/under_test_triplets.json  
  inflating: triplets_dataset/behind_test_triplets.json  
  inflating: triplets_dataset/next to_test_triplets.json  
  inflating: triplets_dataset/on_test_triplets.json  
  inflating: triplets_dataset/under_train_triplets.json  
  inflating: triplets_dataset/behind_train_triplets.json  
  inflating: triplets_dataset/next to_train_triplets.json  
  inflating: triplets_dataset/on_train_triplets.json  
  inflating: triplets_dataset/_DS_Store  
  inflating: new_json_dataset/objects.json  
  inflating: new_json_dataset/predicates.json  
  inflating: new_json_dataset/annotations_train.json  
  inflating: new_json_dataset/annotations_test.json  
  inflating: balancedInputImages.npy  


In [0]:
from itertools import permutations
import seaborn as sns
from keras.datasets import mnist
from sklearn.manifold import TSNE
from sklearn.svm import SVC
import json


In [0]:
with open ('/content/triplets_dataset/behind_train_triplets.json') as json_file:
  behind_train=json.load(json_file)

with open ('/content/triplets_dataset/on_train_triplets.json') as json_file:
  on_train=json.load(json_file)


with open ('/content/triplets_dataset/under_train_triplets.json') as json_file:
  under_train=json.load(json_file)


with open ('/content/triplets_dataset/next to_train_triplets.json') as json_file:
  next_to_train=json.load(json_file)


In [0]:
def create_triplets(behind_train):

  new_dict={}
  for key in behind_train.keys():
    if key=="predicate":
      new_dict[key]=behind_train[key]
      continue
    ex=[]
    val=behind_train[key]
    val=list(val)
    total_length=len(val)
    print(total_length)
    for i in range(int(total_length/2)):
      a_p_n_pairs={}
      a_p_n_pairs["anchor"]=val[i]["filename1"]
      a_p_n_pairs["anchor_details"]=val[i]["positive"]
      a_p_n_pairs["positive"]=val[i+int(total_length/2)]["filename1"]
      a_p_n_pairs["positive_details"]=val[i+int(total_length/2)]["positive"]
      a_p_n_pairs["negative"]=val[i]["filename2"]
      a_p_n_pairs["negative_details"]=val[i]["negative"]
      ex.append(a_p_n_pairs)
      new_dict[key]=ex
  return new_dict
  

In [15]:
behind_train_triplets=create_triplets(behind_train)
print(behind_train_triplets)

1123
{'predicate': 'behind', 'examples': [{'anchor': '9766932005_d859b2b6da_b.jpg', 'anchor_details': {'bbox1': [458, 606, 232, 601], 'bbox2': [230, 489, 102, 781], 'obj1': 'car', 'obj2': 'bus', 'relationship': 'behind'}, 'positive': '5507320891_fdd19ffbbe_b.jpg', 'positive_details': {'bbox1': [173, 617, 524, 640], 'bbox2': [2, 499, 4, 1023], 'obj1': 'building', 'obj2': 'sky', 'relationship': 'behind'}, 'negative': '5041875929_e3b9620600_b.jpg', 'negative_details': {'bbox1': [498, 552, 363, 467], 'bbox2': [472, 575, 441, 677], 'obj1': 'car', 'obj2': 'bus', 'relationship': 'next to'}}, {'anchor': '4874188_684cc9482e_o.jpg', 'anchor_details': {'bbox1': [106, 476, 1, 922], 'bbox2': [13, 206, 564, 677], 'obj1': 'train', 'obj2': 'tree', 'relationship': 'behind'}, 'positive': '7122312237_9bcee2bd5c_o.jpg', 'positive_details': {'bbox1': [34, 611, 225, 478], 'bbox2': [356, 432, 3, 610], 'obj1': 'person', 'obj2': 'counter', 'relationship': 'behind'}, 'negative': '6883648073_b7ce5d4559_b.jpg', '

In [16]:
on_train_triplets=create_triplets(on_train)

2701


In [17]:
under_train_triplets=create_triplets(under_train)

578


In [18]:
next_to_train_triplets=create_triplets(next_to_train)

2096


In [19]:
!wget http://imagenet.stanford.edu/internal/jcjohns/scene_graphs/sg_dataset.zip

--2019-12-02 23:29:02--  http://imagenet.stanford.edu/internal/jcjohns/scene_graphs/sg_dataset.zip
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2000609775 (1.9G) [application/zip]
Saving to: ‘sg_dataset.zip’

sg_dataset.zip      100%[===================>]   1.86G  7.71MB/s    in 4m 27s  

2019-12-02 23:33:29 (7.14 MB/s) - ‘sg_dataset.zip’ saved [2000609775/2000609775]



In [20]:
!unzip sg_dataset.zip

Archive:  sg_dataset.zip
   creating: sg_dataset/
  inflating: sg_dataset/sg_test_annotations.json  
   creating: sg_dataset/sg_test_images/
  inflating: sg_dataset/sg_test_images/10002158834_6dd6c6667f_b.jpg  
  inflating: sg_dataset/sg_test_images/10003466765_d20a7655c6_b.jpg  
  inflating: sg_dataset/sg_test_images/10008170616_a54deca9e3_b.jpg  
  inflating: sg_dataset/sg_test_images/10050248663_2cdb49c115_b.jpg  
  inflating: sg_dataset/sg_test_images/1006083276_0c1a4345fb_o.jpg  
  inflating: sg_dataset/sg_test_images/10085008474_8d72a9dc5e_b.jpg  
  inflating: sg_dataset/sg_test_images/10104268563_965a646010_b.jpg  
  inflating: sg_dataset/sg_test_images/10132007626_6421a783be_b.jpg  
  inflating: sg_dataset/sg_test_images/10132084253_cfa22219a1_b.jpg  
  inflating: sg_dataset/sg_test_images/10142659205_1158853119_b.jpg  
  inflating: sg_dataset/sg_test_images/10142737776_a40de33ab1_b.jpg  
  inflating: sg_dataset/sg_test_images/10161340284_8a72090a5b_b.jpg  
  inflating: sg_data

In [24]:

!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2019-12-02 23:36:00--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2019-12-02 23:36:00--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2019-12-02 23:36:01--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [25]:
!pip install bcolz

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2667807 sha256=9a53ff8ad7e49402e8ce82dc7ce873a961cdf46b3828ceb81069f5252b0eb00d
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [28]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [29]:
import bcolz
import numpy as np
import pickle

words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'6B.50.dat', mode='w')

with open(f'glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
print(vectors.shape)
vectors = bcolz.carray(vectors[1:].reshape((400001, 50)), rootdir=f'6B.50.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'6B.50_idx.pkl', 'wb'))

(20000051,)


In [0]:
vectors = bcolz.open(f'6B.50.dat')[:]
words = pickle.load(open(f'6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [31]:
glove["tree"]

array([ 0.62231 ,  1.1986  , -0.014116,  0.20125 ,  0.69419 ,  0.12068 ,
       -0.90399 , -1.4023  ,  0.43357 , -0.48537 , -0.4645  ,  0.15756 ,
        0.54261 , -0.32467 , -0.025646,  0.45742 ,  0.16561 ,  0.18819 ,
        0.062099, -0.86418 , -1.0425  , -0.81157 ,  0.3126  , -0.20279 ,
        0.55734 , -0.28634 , -0.14874 ,  1.0098  ,  0.25041 , -0.53195 ,
        2.3793  , -0.76966 , -0.63219 ,  0.3203  ,  0.15072 ,  0.23326 ,
       -0.26254 , -0.29461 ,  0.7671  , -0.11577 , -0.68129 , -0.65413 ,
       -0.58914 ,  0.24684 ,  1.5904  ,  0.33025 ,  0.41513 , -1.7468  ,
        0.82453 , -1.0886  ])

In [0]:
import imageio
from PIL import Image

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [0]:
def bbox_mask(img_filename,details):
  img1 = imageio.imread('/content/sg_dataset/sg_train_images/'+img_filename)

  img1 = np.array(img1)
  img1 = img1*0.0
  sub_box=details["bbox1"]
  ob_box=details["bbox2"]
  bboxes = [sub_box[0], sub_box[1], sub_box[2], sub_box[3], ob_box[0], ob_box[1], ob_box[2], ob_box[3]]
  img1[bboxes[0]:bboxes[1], bboxes[2]:bboxes[3]] = 0.5    
  img1[bboxes[4]:bboxes[5], bboxes[6]:bboxes[7]] += 1.0
  img1[bboxes[4]:bboxes[5], bboxes[6]:bboxes[7]] /= 2.0
  img1[bboxes[4]:bboxes[5], bboxes[6]:bboxes[7]] /= 0.75
  img1 = rgb2gray(img1)
  img1 = np.array((img1 * 255).astype(np.uint8))
  img1 = Image.fromarray(img1)
  img1 = img1.resize((100, 100), Image.ANTIALIAS)

  img1 = np.array(img1)
  img1 = img1/255.0
  
  img1 = np.reshape(img1, newshape=[1,img1.shape[0],img1.shape[1],1])

  return img1[0][:,:,0]

In [67]:
import matplotlib.pyplot as plt

X=[]
y=[]

for i in behind_train_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  
  

  anchor_sub=i["anchor_details"]["obj1"]
  anchor_ob=i["anchor_details"]["obj2"]

  positive_sub=i["positive_details"]["obj1"]
  positive_ob=i["positive_details"]["obj2"]

  negative_sub=i["negative_details"]["obj1"]
  negative_ob=i["negative_details"]["obj2"]

  anchors=anchor_sub.split(" ")
  if(len(anchors)>1):
    anchor_sub=anchors[0]

  anchors=anchor_ob.split(" ")
  if(len(anchors)>1):
    anchor_ob=anchors[0]

  positives=positive_sub.split(" ")
  if(len(positives)>1):
    positive_sub=positives[0]

  positives=positive_ob.split(" ")
  if(len(positives)>1):
    positive_ob=positives[0]

  negatives=negative_sub.split(" ")
  if(len(negatives)>1):
    negative_sub=negatives[0]

  negatives=negative_ob.split(" ")
  if(len(negatives)>1):
    negative_ob=negatives[0]

  anchor_sub_vec=glove[anchor_sub]
  anchor_ob_vec=glove[anchor_ob]
  positive_sub_vec=glove[positive_sub]
  positive_ob_vec=glove[positive_ob]
  negative_sub_vec=glove[negative_sub]
  negative_ob_vec=glove[negative_ob]

  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
  
  anchor_final=np.append(anchor_flat,anchor_sub_vec)
  anchor_final=np.append(anchor_final,anchor_ob_vec)
  
  positive_final=np.append(positive_flat,positive_sub_vec)
  positive_final=np.append(positive_final,positive_ob_vec)

  negative_final=np.append(negative_flat,negative_sub_vec)
  negative_final=np.append(negative_final,negative_ob_vec)


  trip=[anchor_final,positive_final,negative_final]
  X.append(trip)
  y.append(2)
  
    

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 11796480 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1365333333 bytes but only got 0. Skipping tag 34464
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 3310. Skipping tag 65535
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 603992064 bytes but only got 0. Skipping tag 9
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  

In [70]:
for i in on_train_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  
  

  anchor_sub=i["anchor_details"]["obj1"]
  anchor_ob=i["anchor_details"]["obj2"]

  positive_sub=i["positive_details"]["obj1"]
  positive_ob=i["positive_details"]["obj2"]

  negative_sub=i["negative_details"]["obj1"]
  negative_ob=i["negative_details"]["obj2"]

  anchors=anchor_sub.split(" ")
  if(len(anchors)>1):
    anchor_sub=anchors[0]

  anchors=anchor_ob.split(" ")
  if(len(anchors)>1):
    anchor_ob=anchors[0]

  positives=positive_sub.split(" ")
  if(len(positives)>1):
    positive_sub=positives[0]

  positives=positive_ob.split(" ")
  if(len(positives)>1):
    positive_ob=positives[0]

  negatives=negative_sub.split(" ")
  if(len(negatives)>1):
    negative_sub=negatives[0]

  negatives=negative_ob.split(" ")
  if(len(negatives)>1):
    negative_ob=negatives[0]

  anchor_sub_vec=glove[anchor_sub]
  anchor_ob_vec=glove[anchor_ob]
  positive_sub_vec=glove[positive_sub]
  positive_ob_vec=glove[positive_ob]
  negative_sub_vec=glove[negative_sub]
  negative_ob_vec=glove[negative_ob]

  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
  
  anchor_final=np.append(anchor_flat,anchor_sub_vec)
  anchor_final=np.append(anchor_final,anchor_ob_vec)
  
  positive_final=np.append(positive_flat,positive_sub_vec)
  positive_final=np.append(positive_final,positive_ob_vec)

  negative_final=np.append(negative_flat,negative_sub_vec)
  negative_final=np.append(negative_final,negative_ob_vec)


  trip=[anchor_final,positive_final,negative_final]
  X.append(trip)
  y.append(0)
  


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 10485760 bytes but only got 0. Skipping tag 55
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6468665344 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 15335424 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 14024704 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expectin

In [71]:
for i in under_train_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  
  

  anchor_sub=i["anchor_details"]["obj1"]
  anchor_ob=i["anchor_details"]["obj2"]

  positive_sub=i["positive_details"]["obj1"]
  positive_ob=i["positive_details"]["obj2"]

  negative_sub=i["negative_details"]["obj1"]
  negative_ob=i["negative_details"]["obj2"]

  anchors=anchor_sub.split(" ")
  if(len(anchors)>1):
    anchor_sub=anchors[0]

  anchors=anchor_ob.split(" ")
  if(len(anchors)>1):
    anchor_ob=anchors[0]

  positives=positive_sub.split(" ")
  if(len(positives)>1):
    positive_sub=positives[0]

  positives=positive_ob.split(" ")
  if(len(positives)>1):
    positive_ob=positives[0]

  negatives=negative_sub.split(" ")
  if(len(negatives)>1):
    negative_sub=negatives[0]

  negatives=negative_ob.split(" ")
  if(len(negatives)>1):
    negative_ob=negatives[0]

  anchor_sub_vec=glove[anchor_sub]
  anchor_ob_vec=glove[anchor_ob]
  positive_sub_vec=glove[positive_sub]
  positive_ob_vec=glove[positive_ob]
  negative_sub_vec=glove[negative_sub]
  negative_ob_vec=glove[negative_ob]

  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
  
  anchor_final=np.append(anchor_flat,anchor_sub_vec)
  anchor_final=np.append(anchor_final,anchor_ob_vec)
  
  positive_final=np.append(positive_flat,positive_sub_vec)
  positive_final=np.append(positive_final,positive_ob_vec)

  negative_final=np.append(negative_flat,negative_sub_vec)
  negative_final=np.append(negative_final,negative_ob_vec)


  trip=[anchor_final,positive_final,negative_final]
  X.append(trip)
  y.append(3)
  

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4278190080 bytes but only got 0. Skipping tag 256
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2097152 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2162688 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to

In [72]:
for i in next_to_train_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  
  

  anchor_sub=i["anchor_details"]["obj1"]
  anchor_ob=i["anchor_details"]["obj2"]

  positive_sub=i["positive_details"]["obj1"]
  positive_ob=i["positive_details"]["obj2"]

  negative_sub=i["negative_details"]["obj1"]
  negative_ob=i["negative_details"]["obj2"]

  anchors=anchor_sub.split(" ")
  if(len(anchors)>1):
    anchor_sub=anchors[0]

  anchors=anchor_ob.split(" ")
  if(len(anchors)>1):
    anchor_ob=anchors[0]

  positives=positive_sub.split(" ")
  if(len(positives)>1):
    positive_sub=positives[0]

  positives=positive_ob.split(" ")
  if(len(positives)>1):
    positive_ob=positives[0]

  negatives=negative_sub.split(" ")
  if(len(negatives)>1):
    negative_sub=negatives[0]

  negatives=negative_ob.split(" ")
  if(len(negatives)>1):
    negative_ob=negatives[0]

  anchor_sub_vec=glove[anchor_sub]
  anchor_ob_vec=glove[anchor_ob]
  positive_sub_vec=glove[positive_sub]
  positive_ob_vec=glove[positive_ob]
  negative_sub_vec=glove[negative_sub]
  negative_ob_vec=glove[negative_ob]

  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
  
  anchor_final=np.append(anchor_flat,anchor_sub_vec)
  anchor_final=np.append(anchor_final,anchor_ob_vec)
  
  positive_final=np.append(positive_flat,positive_sub_vec)
  positive_final=np.append(positive_final,positive_ob_vec)

  negative_final=np.append(negative_flat,negative_sub_vec)
  negative_final=np.append(negative_final,negative_ob_vec)


  trip=[anchor_final,positive_final,negative_final]
  X.append(trip)
  y.append(1)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 11796480 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1365333333 bytes but only got 0. Skipping tag 34464
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 3310. Skipping tag 65535
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 603992064 bytes but only got 0. Skipping tag 9
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  

In [73]:
X_train=np.asarray(X)
y_train=np.asarray(y)
print(X_train.shape)
print(y_train.shape)

(3248, 3, 10100)
(3248,)


In [0]:
import random
p=np.arange(X_train.shape[0])
random.shuffle(p)
X_train_new=X_train[p,:]
y_train_new=y_train[p]

In [75]:
print(X_train_new.shape)
print(y_train_new.shape)

(3248, 3, 10100)
(3248,)


In [0]:
np.save('/content/X_train.npy',X_train_new)
np.save('/content/y_train.npy',y_train_new)

In [0]:
np.save('/content/drive/My Drive/X_train_with_glove.npy',X_train_new)
np.save('/content/drive/My Drive/y_train_with_glove.npy',y_train_new)